# **Fine-tuning XLSR-Wav2Vec2 on Portuguese**

Adapted from work by [Patrick von Platen](https://colab.research.google.com/github/patrickvonplaten/notebooks/blob/master/Fine_Tune_XLSR_Wav2Vec2_on_Turkish_ASR_with_%F0%9F%A4%97_Transformers.ipynb)

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
%%capture
!pip install datasets==1.4.1
!pip install transformers==4.4.0
!pip install torchaudio
!pip install librosa
!pip install jiwer

## Preprocessing the Data, Tokenizer, Feature Extractor

#### Create Wav2Vec2CTCTokenizer

In [3]:
from datasets import load_dataset, load_metric

common_voice_train = load_dataset("common_voice", "pt", split="train+validation")
#common_voice_valid = load_dataset("common_voice", "pt", split="validation")  ## combined train and validation
common_voice_test = load_dataset("common_voice", "pt", split="test")

Couldn't find file locally at common_voice/common_voice.py, or remotely at https://raw.githubusercontent.com/huggingface/datasets/1.4.1/datasets/common_voice/common_voice.py.
The file was picked from the master branch on github instead at https://raw.githubusercontent.com/huggingface/datasets/master/datasets/common_voice/common_voice.py.


Dataset common_voice downloaded and prepared to /root/.cache/huggingface/datasets/common_voice/pt/6.1.0/11f9069cb28d1e9b89bdeb4ad0aabef5bd43213b06f12f3d2349ba39d61968b0. Subsequent calls will reuse this data.


Couldn't find file locally at common_voice/common_voice.py, or remotely at https://raw.githubusercontent.com/huggingface/datasets/1.4.1/datasets/common_voice/common_voice.py.
The file was picked from the master branch on github instead at https://raw.githubusercontent.com/huggingface/datasets/master/datasets/common_voice/common_voice.py.
Reusing dataset common_voice (/root/.cache/huggingface/datasets/common_voice/pt/6.1.0/11f9069cb28d1e9b89bdeb4ad0aabef5bd43213b06f12f3d2349ba39d61968b0)


#### Make the train and test sets

In [4]:
common_voice_train = common_voice_train.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])
#common_voice_valid = common_voice_valid.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])
common_voice_test = common_voice_test.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])

#### Check the data

In [5]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [6]:
show_random_elements(common_voice_train.remove_columns(["path"]), num_examples=20)

,sentence
0,Talvez seja por isso que eles sempre ficam perto de mim.
1,Faça os buracos nas roupas
2,Um cão brinca na neve.
3,Cordeiro assado está cheio
4,Este é o lugar?
5,Jogue fora o conteúdo da alça
6,"Além disso,? neste verão realmente parecia um redemoinho."
7,uma pessoa vestindo uma mochila sentado ao lado de uma montanha rochosa.
8,O estado governante não gerencia.
9,Um seis aparece com mais frequência do que dez.


#### Clean the Data to remove special characters

In [7]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�]'

def remove_special_characters(batch):
    batch["sentence"] = re.sub(chars_to_ignore_regex, '', batch["sentence"]).lower() + " "
    return batch

In [8]:
common_voice_train = common_voice_train.map(remove_special_characters)
#common_voice_valid = common_voice_valid.map(remove_special_characters)
common_voice_test = common_voice_test.map(remove_special_characters)

In [9]:
show_random_elements(common_voice_train.remove_columns(["path"]))

,sentence
0,descobrimos que não precisamos nos destacar
1,o dialeto do norte é fácil de entender
2,ela fala muitas vezes com aspereza
3,as coisas são assim
4,peguei tábuas de madeira de bétula e trabalhei numa estante
5,ambas as experiências reais e eventos imaginários podem ser narrados
6,eu quero manter um quarto
7,um belo pescoço que você tem aí
8,um grupo de homens jogando um jogo de rugby
9,sete anos atrás tudo está chegando


#### Build the Vocabularies

In [10]:
def extract_all_chars(batch):
    all_text = " ".join(batch["sentence"])
    vocab = list(set(all_text))
    return {"vocab": [vocab], "all_text": [all_text]}

In [11]:
vocab_train = common_voice_train.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_train.column_names)
#vocab_valid = common_voice_valid.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_valid.column_names)
vocab_test = common_voice_test.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_test.column_names)

In [12]:
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))

In [13]:
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{' ': 3,
 "'": 33,
 'a': 16,
 'b': 8,
 'c': 1,
 'd': 24,
 'e': 20,
 'f': 5,
 'g': 6,
 'h': 37,
 'i': 36,
 'j': 7,
 'k': 23,
 'l': 10,
 'm': 27,
 'n': 17,
 'o': 9,
 'p': 15,
 'q': 11,
 'r': 29,
 's': 31,
 't': 40,
 'u': 19,
 'v': 39,
 'w': 0,
 'x': 25,
 'y': 38,
 'z': 30,
 'à': 21,
 'á': 41,
 'â': 12,
 'ã': 13,
 'ç': 28,
 'é': 18,
 'ê': 26,
 'í': 32,
 'ñ': 2,
 'ó': 4,
 'ô': 34,
 'õ': 35,
 'ú': 14,
 'ü': 22}

In [14]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [15]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

44

In [16]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

#### Initialize `Wav2Vec2CTCTokenizer` class.

In [17]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

### Create XLSR-Wav2Vec2 Feature Extractor

In [18]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

### Create XLSR-Wav2Vec2 Processor

In [19]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [20]:
processor.save_pretrained("/content/gdrive/MyDrive/COLX585/wav2vec2_pt_processor")

### Preprocess Data

In [21]:
common_voice_train[0]

{'path': '/root/.cache/huggingface/datasets/downloads/extracted/55c28abb37e427173e7a85b9464355d86111a072367ba97d8a00ef5b45f14eeb/cv-corpus-6.1-2020-12-11/pt/clips/common_voice_pt_20464413.mp3',
 'sentence': 'ao treinar um modelo todos os fonemas culturais precisam ser avaliados '}

In [22]:
import torchaudio

def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = torchaudio.load(batch["path"])
    batch["speech"] = speech_array[0].numpy()
    batch["sampling_rate"] = sampling_rate
    batch["target_text"] = batch["sentence"]
    return batch

In [23]:
common_voice_train = common_voice_train.map(speech_file_to_array_fn, remove_columns=common_voice_train.column_names)
#common_voice_valid = common_voice_valid.map(speech_file_to_array_fn, remove_columns=common_voice_valid.column_names)
common_voice_test = common_voice_test.map(speech_file_to_array_fn, remove_columns=common_voice_test.column_names)

### Downsampling

Since the audio files are sampled at 48kHz, we use [`librosa`](https://github.com/librosa/librosa) library to downsample.

In [24]:
import librosa
import numpy as np

def resample(batch):
    batch["speech"] = librosa.resample(np.asarray(batch["speech"]), 48_000, 16_000)
    batch["sampling_rate"] = 16_000
    return batch

In [25]:
common_voice_train = common_voice_train.map(resample, num_proc=4)
#common_voice_valid = common_voice_valid.map(resample, num_proc=8)
common_voice_test = common_voice_test.map(resample, num_proc=4)

In [26]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(common_voice_train)-1)

ipd.Audio(data=np.asarray(common_voice_train[rand_int]["speech"]), autoplay=True, rate=16000)

In [27]:
rand_int = random.randint(0, len(common_voice_train)-1)

print("Target text:", common_voice_train[rand_int]["target_text"])
print("Input array shape:", np.asarray(common_voice_train[rand_int]["speech"]).shape)
print("Sampling rate:", common_voice_train[rand_int]["sampling_rate"])

Target text: seus registros estavam incompletos 
Input array shape: (65280,)
Sampling rate: 16000


In [28]:
def prepare_dataset(batch):
    # check that all files have the correct sampling rate
    assert (
        len(set(batch["sampling_rate"])) == 1
    ), f"Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}."

    batch["input_values"] = processor(batch["speech"], sampling_rate=batch["sampling_rate"][0]).input_values
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["target_text"]).input_ids
    return batch

In [29]:
common_voice_train = common_voice_train.map(prepare_dataset, remove_columns=common_voice_train.column_names, batch_size=8, num_proc=8, batched=True)
#common_voice_valid = common_voice_valid.map(prepare_dataset, remove_columns=common_voice_valid.column_names, batch_size=8, num_proc=8, batched=True)
common_voice_test = common_voice_test.map(prepare_dataset, remove_columns=common_voice_test.column_names, batch_size=8, num_proc=8, batched=True)

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


## Training

### Set-up Trainer
The code for the data collator was copied from [this example](https://github.com/huggingface/transformers/blob/9a06b6b11bdfc42eea08fa91d0c737d1863c99e3/examples/research_projects/wav2vec2/run_asr.py#L81).

In [30]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [31]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

### Define WER metric

In [32]:
wer_metric = load_metric("wer")

In [33]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [77]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-xlsr-53", 
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    gradient_checkpointing=True, 
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-xlsr-53 and are newly initialized: ['lm_head.weight', 'lm_head.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [78]:
model.freeze_feature_extractor()

In [36]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="/content/gdrive/MyDrive/COLX585/wav2vec2_pt",
  # output_dir="./wav2vec2-large-xlsr-turkish-demo",
  group_by_length=True,
  per_device_train_batch_size=16,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=30,
  fp16=True,
  save_steps=800,
  eval_steps=400,
  logging_steps=400,
  learning_rate=3e-4,
  warmup_steps=500,
  save_total_limit=2,
)

In [80]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=common_voice_train,
    eval_dataset=common_voice_test,
    tokenizer=processor.feature_extractor,

)

### Training

Training was interrupted at step 8800, so this shows the trainer resuming from that checkpoint.

In [38]:
trainer.train("/content/gdrive/MyDrive/COLX585/wav2vec2_pt/checkpoint-8800")

Step,Training Loss,Validation Loss,Wer,Runtime,Samples Per Second
9200,1.016900,0.357532,0.295837,602.957000,7.697000
9600,0.053300,0.331755,0.290308,610.036100,7.608000
10000,0.048500,0.328773,0.287160,607.344700,7.641000
10400,0.042700,0.326642,0.286026,609.727300,7.612000


TrainOutput(global_step=10410, training_loss=0.044643297502332646, metrics={'train_runtime': 11660.668, 'train_samples_per_second': 0.893, 'total_flos': 4.52704787010419e+19, 'epoch': 30.0, 'init_mem_cpu_alloc_delta': 347756, 'init_mem_gpu_alloc_delta': 1261935616, 'init_mem_cpu_peaked_delta': 18306, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 4012356, 'train_mem_gpu_alloc_delta': 5090804736, 'train_mem_cpu_peaked_delta': 468610769, 'train_mem_gpu_peaked_delta': 7833652736})

#### Save model

In [43]:
model = Wav2Vec2ForCTC.from_pretrained("/content/gdrive/MyDrive/COLX585/wav2vec2_pt/checkpoint-10400").to("cuda")
processor = Wav2Vec2Processor.from_pretrained("/content/gdrive/MyDrive/COLX585/wav2vec2_pt_processor")

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


#### Decoding an example

In [66]:
input_dict = processor(common_voice_test[-1]["input_values"], return_tensors="pt", padding=True)

logits = model(input_dict.input_values.to("cuda")).logits

pred_ids = torch.argmax(logits, dim=-1)[0]

It is strongly recommended to pass the ``sampling_rate`` argument to this function.Failing to do so can result in silent errors that might be hard to debug.


In [ ]:
common_voice_test[1]["input_values"]

In [45]:
common_voice_test_transcription = load_dataset("common_voice", "pt", data_dir="./cv-corpus-6.1-2020-12-11", split="test")

Couldn't find file locally at common_voice/common_voice.py, or remotely at https://raw.githubusercontent.com/huggingface/datasets/1.4.1/datasets/common_voice/common_voice.py.
The file was picked from the master branch on github instead at https://raw.githubusercontent.com/huggingface/datasets/master/datasets/common_voice/common_voice.py.
Using custom data configuration pt-ad9f7b76efa9f3a0
Reusing dataset common_voice (/root/.cache/huggingface/datasets/common_voice/pt-ad9f7b76efa9f3a0/6.1.0/11f9069cb28d1e9b89bdeb4ad0aabef5bd43213b06f12f3d2349ba39d61968b0)


In [68]:
print("Prediction:")
print(processor.decode(pred_ids))

print("\nReference:")
print(common_voice_test_transcription[-1]["sentence"].lower())


Prediction:
atualmente o readial não pôde encontrar simos nazaturma

Reference:
actualmente, o horário de aula não pode encontrar as informações da turma.


In [58]:
processor.decode(pred_ids)

'e dia genheiro emprestado das pessoas da aldeia'

### Creating the Prediction Files

In [69]:
len(common_voice_test)

4641

In [ ]:
with open("pt_preds.txt", "w") as f:
      for i in range(len(common_voice_test)):
        input_dict = processor(common_voice_test[i]["input_values"], return_tensors="pt", padding=True)
        logits = model(input_dict.input_values.to("cuda")).logits
        pred_ids = torch.argmax(logits, dim=-1)[0]
        pred = processor.decode(pred_ids)
        f.write(pred + "\n")

In [71]:
with open("pt_gold.txt", "w") as f:
      for i in range(len(common_voice_test_transcription)):
        gold = common_voice_test_transcription[i]["sentence"].lower()
        f.write(gold + "\n")